# 3D Brain Visualizations

## 3D topographic brain map

The 3D topographic brain map provides a view of voltage measurements as a heatmap converted to estimated position on the brain. There are 2 plot options with different backends: 

1) [matplotlib](#matplotlib) (the version found on the UI)

2) [pyVista](#pyvista) (a better rendering but incompatible with the UI)

Both plots can be generated as an animation to view changes over time or as a standalone plot.

![](instruction_imgs/presentattion_brain.gif)

## General Setup
### Import required modules

In [1]:
from simpl_eeg import topomap_3d_brain, eeg_objects

In [2]:
import warnings
warnings.filterwarnings('ignore')

```{note}
Please include the line below in your IDE so that the changes would be simultaneously reflected when you make a change to the python scripts.
```

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

### Create epoched data
For additional options see [Creating EEG Objects](eeg_objects.html#intro) section.

In [5]:
experiment_folder = "../../data/109"
nth_epoch = 0

epochs = eeg_objects.Epochs(experiment_folder)
epoch = epochs.get_nth_epoch(nth_epoch)

Reading /Users/sasha/mds/simpl_eeg_capstone/data/109/fixica.fdt


Not setting metadata


Not setting metadata


33 matching events found


Setting baseline interval to [-0.2998046875, 0.0] sec


Applying baseline correction (mode: mean)


0 projection items activated


Loading data for 33 events and 2049 original time points ...


0 bad epochs dropped


### Generate forward and inverse (optional)

```{note}
- Before an animation or plot can be generated, a **"forward"** and **"inverse"** (abbreviated as **"stc"**) must first be generated. If they are not provided to either of the plotting animations they will be automatically generated, **HOWEVER** this will increase the time it takes to generate the figure.

- The forward/inverse are used to retrieve a brain model to attach the EEG data and to do some of the mapping calculations. The forward downloads 'fsaverage' MRI data which represents a brain averaged out from dozens of different patients.
```

#### Generate Forward

In [6]:
fwd = topomap_3d_brain.create_fsaverage_forward(epoch)

0 files missing from root.txt in /Users/sasha/mne_data/MNE-fsaverage-data


0 files missing from bem.txt in /Users/sasha/mne_data/MNE-fsaverage-data/fsaverage


Source space          : /Users/sasha/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif


MRI -> head transform : /opt/miniconda3/lib/python3.8/site-packages/mne/data/fsaverage/fsaverage-trans.fif


Measurement data      : instance of Info


...

    Skipping interior check for 2241 sources that fit inside a sphere of radius   47.7 mm


    Skipping solid angle check for 0 points using Qhull


Setting up for EEG...


Computing EEG at 20484 source locations (free orientations)...


Finished.


#### Generate Inverse

In [ ]:
stc = topomap_3d_brain.create_inverse_solution(epoch, fwd)

<a id="matplotlib"></a>
## Create a matplotlib 3D brain animation (recommended)

### Define parameters

A detailed description of all parameters can be found in the `topomap_3d_brain.animate_matplot_brain` docstring:

In [ ]:
help(topomap_3d_brain.animate_matplot_brain)

In [ ]:
colormap = "RdBu_r"

#### Generate animation with matplotlib backend (slow but recommended)

In [ ]:
%%capture

matplotlib_animation = topomap_3d_brain.animate_matplot_brain(epoch, stc = stc, views = 'lat', hemi = 'lh')

from IPython.display import HTML
video = HTML(matplotlib_animation.to_jshtml())

In [ ]:
video

### Saving the animation

#### Save as gif

```python
anim_brain = topomap_3d_brain.animate_matplot_brain(epoch, stc = stc, views = 'lat', hemi = 'lh')

gif_file_path = "examples/topomap_3d_brain.gif" 
anim_brain.save(gif_file_path, fps=5, dpi=300)
```

#### Save as mp4

```python
mp4_file_path = "examples/topo_2d.mp4"
anim_brain.save(mp4_file_path, fps=5, dpi=300)
```

```{note}
If `FFMpegWriter` does not work on your computer you can save the file as a gif first and then convert it into mp4 file by running the code below.
```
```python
import moviepy.editor as mp

clip = mp.VideoFileClip(gif_file_path)
clip.write_videofile(mp4_file_path)
```

## Create a matplotlib 3D brain figure (recommended)

### Generating a matplotlib plot

In [ ]:
%%capture
matplot_brain_fig = topomap_3d_brain.plot_topomap_3d_brain(epoch, stc=stc, backend='matplotlib')

### Save the plot
You can change the file to different formats by changing the format argument in the function. It supports `png`, `pdf`, `svg`.
```python
file_path = "examples/topomap_3d_brain.svg"  
matplot_brain_fig.savefig(file_path, format='svg')
```

<a id="pyvista"></a>
## Create a pyVista 3D brain animation

### Generate figure with pyvista backend (NOT CURRENTLY WORKING)

In [ ]:
#pyvista_brain_fig = topomap_3d_brain.plot_topomap_3d_brain(epoch, stc = stc, backend = 'pyvista')

### Save animation with pyvista backend (NOT CURRENTLY WORKING)

In [ ]:
#topomap_3d_brain.save_animated_topomap_3d_brain(pyvista_brain_fig, filename = "brain_animation.gif")